# Введение
Перед вами небольшой класс – реализация [модели pinhole камеры использующейся в OpenCV](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html).
Чтобы было нагляднее вся математика написана напрямую. В дз лучше использовать методы OpenCV (например `cv2.projectPoints`).

In [ ]:
import numpy as np


class OpenCvCamera:
    def __init__(self, f, resolution_px):
        # fx == fx == f. Во всех камерах что мы будем использовать физические пиксели всегда будут квадратные
        self.f = f 
        
        # В нашем случае resolution это размеры изображений, которые соответствуют этой камере
        # img.shape == (self.resolution_y, self.resolution_x)
        self.resolution_x, self.resolution_y = resolution_px
        
        # Во всех изображениях что мы будем использовать центр изображения, 
        # через который проходит оптическая ось камеры, всегда будет в центральном пикселе
        # Это может быть не так если, например, мы бы работали с обрезанным изображением
        self.cx = self.resolution_x / 2
        self.cy = self.resolution_y / 2
    
    def proj(self, p):
        # возвращает 2d проекцию 3D точки `p`
        return np.array([self.f * p[0] / p[2] + self.cx, self.f * p[1] / p[2] + self.cy])

    def __str__(self):
        return ('OpenCvCamera [fx = fx = %.1f px, cx = %.1f px, cy = %.1f px, resolution = %d x %d px]' % 
                (self.f, self.cx, self.cy, self.resolution_x, self.resolution_y))


# Задание 1
Напишите функцию `camera_from_physical_parameters`, которая по заданным параметрам создает соответствующий объект класса `OpenCvCamera`.

Так как у нас `fx == fx == f` ожидается что соотношение сторон сенсора и изображения одинаковы.

In [ ]:
def camera_from_physical_parameters(focal_length_mm: float, sensor_wh_mm: tuple, resolution_px: tuple):
    pass


assert(np.isclose(camera_from_physical_parameters(1, (1, 1), (100, 100)).f, 100))

Перед вами пример типичной full-frame камеры. 35mm full-frame это один из [популярных размеров сенсора](https://en.wikipedia.org/wiki/Image_sensor_format#Table_of_sensor_formats_and_sizes).

Это камера с сенсором 35mm full-frame размера `36x24мм`, разрешением фотографий `6240x4160` пикселей, и объективом с выставленным значением фокусного расстояния `45` мм.

Например камера [Canon EOS 6D Mark II](https://en.wikipedia.org/wiki/Canon_EOS_6D_Mark_II) с объективом [Canon EF 24-70 мм](https://en.wikipedia.org/wiki/Canon_EF_24–70mm_lens)

In [ ]:
canon_6d_mark_ii_45 = camera_from_physical_parameters(45, (36, 24), (6240, 4160))
print(canon_6d_mark_ii_45)

print('Допустим все 3d координаты в метрах')
print('Точка в 3 метрах перед камерой будет в центральном пикселе %s' % canon_6d_mark_ii_45.proj((0, 0, 3)))
print('Точка в 3 метрах перед камерой и 1 метром правее '
      'будет в пикселе %s. На границе изображения' % canon_6d_mark_ii_45.proj((1, 0, 3)))
print('А точка в 3 метрах перед камерой и 1 метром выше '
      'будет в пикселе %s. Вне изображения' % canon_6d_mark_ii_45.proj((0, -1, 3)))

Еще один пример. Камера iPhone Xs.

In [ ]:
iphone_xs_main_camera = camera_from_physical_parameters(4.25, (5.64, 4.23), (4032, 3024))
print(iphone_xs_main_camera)

print('Точка в 3 метрах перед камерой и 1 метром правее '
      'будет в пикселе %s. Справа в центре' % iphone_xs_main_camera.proj((1, 0, 3)))
print('А точка в 3 метрах перед камерой и 1 метром выше '
      'будет в пикселе %s. Сверху в центре' % iphone_xs_main_camera.proj((0, -1, 3)))

# Задание 2
Напишите функцию `compute_camera_fov`, которая для заданной `camera` возвращает горизонтальный и вертикальный FoV (field of view) в радианах.

In [ ]:
def compute_camera_fov(camera: OpenCvCamera):
    pass

_test_camera = camera_from_physical_parameters(0.5, (1, 1), (100, 100))
assert(np.all(np.isclose(np.degrees(compute_camera_fov(_test_camera)), (90, 90))))

print('canon_6d_mark_ii_45 (fov_x, fov_x) = %s degrees' % np.degrees(compute_camera_fov(canon_6d_mark_ii_45)))
print('iphone_xs_main_camera (fov_x, fov_x) = %s degrees' % np.degrees(compute_camera_fov(iphone_xs_main_camera)))

# Задание 3
Напишите функцию `compute_camera_fl`, обратную `compute_camera_fov`. 
Функция `compute_camera_fl` принимает на вход `sensor_wh_mm` и `fov` в радианах и возвращает два значения `fl` чтобы получить заданный `fov` по горизонтали и вертикали соответственно.

In [ ]:
def compute_camera_fl(sensor_wh_mm: tuple, fov: float):
    pass

assert(np.all(np.isclose(compute_camera_fl((1, 1), np.radians(90)), (0.5, 0.5))))

print('Для того чтобы на fullframe камере получить горизонтальное '
      'поле зрения как у iPhone Xs\nнужно использовать объектив с '
      'фокусным расстоянием %f мм' % compute_camera_fl((36, 24), np.radians(67.13))[0])
print('Для того чтобы на fullframe камере получить вертикальное '
      'поле зрения как у iPhone Xs\nнужно использовать объектив с '
      'фокусным расстоянием %f мм' % compute_camera_fl((36, 24), np.radians(52.91419765))[1])

То что мы посчитали выше называется [35мм эквивалентным фокусным расстоянием](https://en.wikipedia.org/wiki/35_mm_equivalent_focal_length). Можно считать по горизонтали, вертикали или по диагонали. При разном соотношении сторон значения будут разные.

Так в своей рекламе Apple писали "The iPhone XS wide-angle lens has an equivalent focal length of 26mm".

# Задание 4
Напишите функцию `camera_from_fov`, которая по заданным параметрам создает соответствующий объект класса `OpenCvCamera`.

In [ ]:
def camera_from_fov(fov_x: float, resolution_px: tuple):
    pass

assert(np.isclose(camera_from_fov(np.radians(90), (100, 300)).f, 50))

# Задание 5
Напишите функции
- `transform` – трансформирует 3D точку с помощью заданной трансформации (матрицы поворота и параллельного переноса);
- `compose_transforms` – возвращает трансформацию (пару из матрицы поворота и параллельного переноса), эквиванентную последовательному применению двух заданных трансформаций;
- `inverse_transform` – возвращает трансформацию, обратную заданной (тоесть при применений `transform` сначала исходной трансформацией, потом этой, точка останеться на месте).


In [ ]:
def transform(point_3d: np.array, rot_mat: np.array, translation: np.array):
    pass

def compose_transforms(rot_mat1: np.array, translation1: np.array, rot_mat2: np.array, translation2: np.array):
    pass

def inverse_transform(rot_mat: np.array, translation: np.array):
    pass


def test_transform_functions(point_3d):
    zero_rot = np.eye(3)
    zero_t = np.zeros(3)
    assert(np.all(np.isclose(transform(point_3d, zero_rot, zero_t), point_3d)))
    
    composed_rot, composed_t = compose_transforms(zero_rot, zero_t, zero_rot, zero_t)
    assert(np.all(np.isclose(composed_rot, zero_rot)))
    assert(np.all(np.isclose(composed_t, zero_t)))
    
    test_rot = np.array([[np.cos(-1), -np.sin(-1), 0], 
                         [np.sin(-1), np.cos(-1), 0],
                         [0, 0, 1]])
    test_t = np.array([92, 42, -21])
    
    composed_rot, composed_t = compose_transforms(zero_rot, zero_t, test_rot, test_t)
    assert(np.all(np.isclose(composed_rot, test_rot)))
    assert(np.all(np.isclose(composed_t, test_t)))
    
    composed_rot, composed_t = compose_transforms(test_rot, test_t, zero_rot, zero_t)
    assert(np.all(np.isclose(composed_rot, test_rot)))
    assert(np.all(np.isclose(composed_t, test_t)))

    point_3d_transformed = transform(point_3d, test_rot, test_t)
    test_rot_inv, test_t_inv = inverse_transform(test_rot, test_t)
    assert(np.all(np.isclose(transform(point_3d_transformed, test_rot_inv, test_t_inv), point_3d)))
    
    test_2_rot, test_2_t = compose_transforms(test_rot, test_t, test_rot, test_t)
    point_3d_transformed_2 = transform(point_3d, test_2_rot, test_2_t)
    assert(np.all(np.isclose(transform(point_3d_transformed_2, test_rot_inv, test_t_inv), point_3d_transformed)))

    
test_transform_functions(np.array([0, 0, 0]))
test_transform_functions(np.array([0, 1, 0]))
test_transform_functions(np.array([-20, 22, 33]))

# Задание 6
Есть `camera` с известными параметрами. С помощью нее сделаны две фотографии одной 3D точки из разных 3D позиций `camera_rot1` и `camera_pos1`, `camera_rot2` и `camera_pos2`. 
3D точка попала в пиксель `image_point_1` в первом и в пиксель `image_point_2` во втором изображении.

Функция `triangilate` должна вернуть 3D позицию этой 3D точки.


*В тестах нету поворотов камеры*

In [ ]:
def triangilate(camera: OpenCvCamera,
                camera_rot1: np.array, camera_pos1: np.array,
                camera_rot2: np.array, camera_pos2: np.array,
                image_point_1: np.array, image_point_2: np.array):
    pass


In [ ]:
def test_triangulate(camera: OpenCvCamera,
                     camera_pos1: np.array, camera_pos2: np.array,
                     point_pos: np.array):
    image_point_1 = camera.proj(point_pos - camera_pos1)
    image_point_2 = camera.proj(point_pos - camera_pos2)
    found_point_pos = triangilate(camera, 
                                  np.eye(3), camera_pos1, 
                                  np.eye(3), camera_pos2,
                                  image_point_1, image_point_2)
    print('found position: %s' % found_point_pos)
    assert(np.all(np.isclose(point_pos, found_point_pos)))
    
test_triangulate(canon_6d_mark_ii_45,
                 np.array([0, 0, 0]), np.array([0, 0.5, 0]),
                 np.array([0, 0, 3]))
test_triangulate(canon_6d_mark_ii_45,
                 np.array([0, 0, 0]), np.array([0, 0.5, 0]),
                 np.array([0, 0.1, 3]))
test_triangulate(canon_6d_mark_ii_45,
                 np.array([0, 0.3, -3]), np.array([0.3, 0.5, -2]),
                 np.array([0.213, 0.211, 4.312]))

# Этот тест не должен работать. Линии, соединяющие 3D точку с центрами камер параллельны. 
# Триангуляция в данном случае не возможна.
# test_triangulate(canon_6d_mark_ii_45,
#                  np.array([0, 0, -3]), np.array([0, 0, -2]),
#                  np.array([0, 0, 4]))

# Задание 7
Есть `camera` с известными параметрами. С помощью нее сделана фотография нескольких 3D точек.
3D точки `point_cloud` попали в пиксели `image_points`.

Вы знаете что поворот камеры нулевой (`R = I`). 

Функция `find_camera_position` должна вернуть 3D позицию `camera` (вектор `t`), из которой при проекции точек `point_cloud` они попадут в пиксели `image_points`.

*Подсказка: `np.linalg.lstsq`*

In [ ]:
def find_camera_position(camera: OpenCvCamera, point_cloud: np.array, image_points: np.array):
    pass



def find_camera_position_test(camera, camera_pos):
    N = 20
    point_cloud = np.array([[p_idx / 2, (p_idx - N / 2) / 2, 10 * (p_idx / N) ** 2 + 20] for p_idx in range(N)])
    image_points = np.array([camera.proj(point_cloud[p_idx, :] - camera_pos) for p_idx in range(N)])
    found_pos = find_camera_position(camera, point_cloud, image_points)
    assert(np.all(np.isclose(camera_pos, found_pos)))

find_camera_position_test(canon_6d_mark_ii_45, np.array([0, 0, 0]))
find_camera_position_test(canon_6d_mark_ii_45, np.array([0, 0, 1]))
find_camera_position_test(canon_6d_mark_ii_45, np.array([-2, 0.231, -3.123]))

# Задание 8*
Есть `camera` с неизвестными параметрами (неточным `camera.f`). С помощью нее сделана фотография нескольких 3D точек.
3D точки `point_cloud` попали в пиксели `image_points`.

Вы знаете что поворот камеры нулевой (`R = I`). 

Функция `find_camera_position_and_internal_parameters` должна вернуть искомую 3D позицию `camera` (вектор `t`) и искомое значение `camera.f`.

In [ ]:
def find_camera_position_and_internal_parameters(camera: OpenCvCamera, point_cloud: np.array, image_points: np.array):
    pass



def find_camera_position_and_internal_parameters_test(camera, camera_pos):
    N = 20
    point_cloud = np.array([[p_idx / 2, (p_idx - N / 2) / 2, 10 * (p_idx / N) ** 2 + 20] for p_idx in range(N)])
    image_points = np.array([camera.proj(point_cloud[p_idx, :] - camera_pos) for p_idx in range(N)])
    optimal_f = camera.f
    camera.f *= 1.2
    found_pos, found_f = find_camera_position_and_internal_parameters(camera, point_cloud, image_points)
    camera.f = optimal_f # just in case
    assert(np.all(np.isclose(camera_pos, found_pos, atol=0.5)))
    assert(np.all(np.isclose(optimal_f, found_f, atol=optimal_f/10)))

find_camera_position_and_internal_parameters_test(canon_6d_mark_ii_45, np.array([0, 0, 0]))
find_camera_position_and_internal_parameters_test(canon_6d_mark_ii_45, np.array([0, 0, 1]))
find_camera_position_and_internal_parameters_test(canon_6d_mark_ii_45, np.array([-2, 0.231, -3.123]))